In [153]:
import pandas as pd
import pprint
import ast
import json
import numpy as np
from scipy.stats import entropy
from scipy.special import softmax
import numpy as np
import pandas as pd
from scipy.special import softmax
from scipy.stats import entropy
import ast
import os


In [154]:

def calculate_metric_with_true_label(df):
    

    def parse_top_logprobs(val):
        if isinstance(val, str):
            try:
                return ast.literal_eval(val)
            except Exception as e:
                print(f"❌ Nie udało się sparsować: {val[:60]}... ➡ {e}")
                return None
        return val

    df['top_logprobs'] = df['top_logprobs'].apply(parse_top_logprobs)
    
    df['logprobs_cal'] = df['top_logprobs'].apply(
    lambda x: np.array([item['logprob'] for item in x if isinstance(item, dict) and 'logprob' in item])
    if isinstance(x, list) else np.array([])
)



    # Przekształcenie logprobs do prawdopodobieństw przez softmax
    df['probs'] = df['logprobs_cal'].apply(
        lambda logprobs: softmax(logprobs) if len(logprobs) > 0 else np.array([])
    )

    # Entropia rozkładu
    df['entropy'] = df['probs'].apply(
        lambda probs: entropy(probs, base=2) if len(probs) > 0 else np.nan
    )

    # Margin: różnica między top-1 a top-2
    df['margin'] = df['probs'].apply(
        lambda probs: np.sort(probs)[-1] - np.sort(probs)[-2] if len(probs) > 1 else np.nan
    )

    # KL divergence względem idealnego rozkładu (1.0 na poprawnej etykiecie)
    def kl_div(probs, top_logprobs, label):
        if not isinstance(top_logprobs, list) or len(probs) == 0:
            return np.nan

        try:
            # lista tokenów jako stringi
            tokens = [str(item['token']).strip().lower() for item in top_logprobs if isinstance(item, dict) and 'token' in item]
            label_str = str(label).strip().lower()

            if label_str not in tokens:
                return np.nan

            match_idx = tokens.index(label_str)
            ideal_dist = np.zeros_like(probs)
            ideal_dist[match_idx] = 1.0

            return entropy(ideal_dist, probs, base=2)

        except Exception as e:
            print(f"⚠️ Błąd w kl_div: {e}")
            return np.nan


    # df['kl_divergence'] = df.apply(
    #     lambda row: kl_div(row['probs'], row['top_logprobs'], row[column_name]), axis=1
    # )
    
    df['kl_divergence'] = df['probs'].apply(
    lambda probs: np.sum(probs * np.log(probs / (1 / len(probs)))) if len(probs) > 0 else np.nan
        )
    
    # df['top_prob'] = df['probs'].apply(lambda p: np.max(p) if len(p) > 0 else np.nan)
    # df['kl_divergence'] = 2 * (1 - df['top_prob'])



    # Normalizacja (min-max)
    for col in ['entropy', 'margin', 'kl_divergence']:
        norm_col = f'normalized_{col}'
        min_val = df[col].min()
        max_val = df[col].max()
        if pd.notnull(min_val) and pd.notnull(max_val) and max_val > min_val:
            df[norm_col] = (df[col] - min_val) / (max_val - min_val)
        else:
            df[norm_col] = 0.0

    return df




In [155]:
model = 'gpt4omini'
# model = 'llama3'
# model = 'cohere'
dataset = 'go_emotions'

path1 =  '../NO_EXAMPLES/results/no_samples_numbers_temp0'
path2 = f'/{model}/{dataset}_{model}_no_samples_numbers_temp0_exp1.csv'
df = pd.read_csv(path1 + path2)

In [156]:
df

,text,output,logprobs,top_logprobs,original_label
0,Ok.... he can *call* it the State of the Union...,0,"{'content': [{'token': '0', 'bytes': [48], 'lo...","[{'token': '0', 'bytes': [48], 'logprob': -0.0...",disapproval
1,Wheres the telescope?,1,"{'content': [{'token': '1', 'bytes': [49], 'lo...","[{'token': '1', 'bytes': [49], 'logprob': -3.4...",curiosity
2,> Why create more jobs when no one is availabl...,1,"{'content': [{'token': '1', 'bytes': [49], 'lo...","[{'token': '1', 'bytes': [49], 'logprob': -0.1...",confusion
3,[NAME] deserves to play for a proper club ffs.,0,"{'content': [{'token': '0', 'bytes': [48], 'lo...","[{'token': '0', 'bytes': [48], 'logprob': -0.0...",admiration
4,"I'm sorry, do people NOT listen to [NAME] 9th ...",4,"{'content': [{'token': '4', 'bytes': [52], 'lo...","[{'token': '4', 'bytes': [52], 'logprob': -1.2...",gratitude
...,...,...,...,...,...
1915,"Yeah, i dont know bout you guys, but my parent...",4,"{'content': [{'token': '4', 'bytes': [52], 'lo...","[{'token': '4', 'bytes': [52], 'logprob': -0.2...",approval
1916,This is hilarious! Sounds similar to my buddy'...,17,"{'content': [{'token': '17', 'bytes': [49, 55]...","[{'token': '17', 'bytes': [49, 55], 'logprob':...",amusement
1917,But that jumper looks hella cosy,14,"{'content': [{'token': '14', 'bytes': [49, 52]...","[{'token': '14', 'bytes': [49, 52], 'logprob':...",joy
1918,Sorry. This thing knows whats up!,1,"{'content': [{'token': '1', 'bytes': [49], 'lo...","[{'token': '1', 'bytes': [49], 'logprob': -0.0...",remorse


In [157]:
df_metric = calculate_metric_with_true_label(df)
df_metric

,text,output,logprobs,top_logprobs,original_label,logprobs_cal,probs,entropy,margin,kl_divergence,normalized_entropy,normalized_margin,normalized_kl_divergence
0,Ok.... he can *call* it the State of the Union...,0,"{'content': [{'token': '0', 'bytes': [48], 'lo...","[{'token': '0', 'bytes': [48], 'logprob': -0.0...",disapproval,"[-0.031251467764377594, -3.9062514305114746, -...","[0.9692320274730504, 0.020115769725557522, 0.0...",0.239857,0.949116,2.829476,0.071226,0.949116,0.928774
1,Wheres the telescope?,1,"{'content': [{'token': '1', 'bytes': [49], 'lo...","[{'token': '1', 'bytes': [49], 'logprob': -3.4...",curiosity,"[-3.47626737493556e-05, -10.87503433227539, -1...","[0.9999652938140706, 1.89248577271334e-05, 7.8...",0.000632,0.999946,2.995294,0.000188,0.999946,0.999812
2,> Why create more jobs when no one is availabl...,1,"{'content': [{'token': '1', 'bytes': [49], 'lo...","[{'token': '1', 'bytes': [49], 'logprob': -0.1...",confusion,"[-0.13834109902381897, -3.013341188430786, -3....","[0.8708748633824798, 0.04913139339022995, 0.02...",0.899396,0.821743,2.372318,0.267079,0.821743,0.732921
3,[NAME] deserves to play for a proper club ffs.,0,"{'content': [{'token': '0', 'bytes': [48], 'lo...","[{'token': '0', 'bytes': [48], 'logprob': -0.0...",admiration,"[-0.0009293294278904796, -7.500929355621338, -...","[0.9990712125211695, 0.0005525706578363728, 0....",0.012025,0.998519,2.987397,0.003571,0.998519,0.996429
4,"I'm sorry, do people NOT listen to [NAME] 9th ...",4,"{'content': [{'token': '4', 'bytes': [52], 'lo...","[{'token': '4', 'bytes': [52], 'logprob': -1.2...",gratitude,"[-1.2217153310775757, -1.5967153310775757, -1....","[0.29482515640398504, 0.20263016911433046, 0.1...",2.762215,0.092195,1.081111,0.820250,0.092195,0.179750
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915,"Yeah, i dont know bout you guys, but my parent...",4,"{'content': [{'token': '4', 'bytes': [52], 'lo...","[{'token': '4', 'bytes': [52], 'logprob': -0.2...",approval,"[-0.23121221363544464, -1.6062122583389282, -6...","[0.7935755070987813, 0.20064730148576904, 0.00...",0.788371,0.592928,2.449275,0.234110,0.592928,0.765890
1916,This is hilarious! Sounds similar to my buddy'...,17,"{'content': [{'token': '17', 'bytes': [49, 55]...","[{'token': '17', 'bytes': [49, 55], 'logprob':...",amusement,"[-0.0004427051462698728, -7.7504425048828125, ...","[0.9995574022312064, 0.00043055198111212393, 1...",0.005654,0.999127,2.991813,0.001679,0.999127,0.998321
1917,But that jumper looks hella cosy,14,"{'content': [{'token': '14', 'bytes': [49, 52]...","[{'token': '14', 'bytes': [49, 52], 'logprob':...",joy,"[-0.07127564400434494, -3.6962757110595703, -4...","[0.9313384367361074, 0.02481932698940406, 0.01...",0.551327,0.906519,2.613582,0.163719,0.906519,0.836281
1918,Sorry. This thing knows whats up!,1,"{'content': [{'token': '1', 'bytes': [49], 'lo...","[{'token': '1', 'bytes': [49], 'logprob': -0.0...",remorse,"[-0.09035232663154602, -2.5903522968292236, -4...","[0.9136364910345172, 0.07499585234436261, 0.00...",0.497019,0.838641,2.651225,0.147592,0.838641,0.852408


In [158]:
output_path = 'entropy' + path2
os.makedirs(os.path.dirname(output_path), exist_ok=True)

In [159]:
df_metric[['text', 'original_label', 'normalized_entropy']]

,text,original_label,normalized_entropy
0,Ok.... he can *call* it the State of the Union...,disapproval,0.071226
1,Wheres the telescope?,curiosity,0.000188
2,> Why create more jobs when no one is availabl...,confusion,0.267079
3,[NAME] deserves to play for a proper club ffs.,admiration,0.003571
4,"I'm sorry, do people NOT listen to [NAME] 9th ...",gratitude,0.820250
...,...,...,...
1915,"Yeah, i dont know bout you guys, but my parent...",approval,0.234110
1916,This is hilarious! Sounds similar to my buddy'...,amusement,0.001679
1917,But that jumper looks hella cosy,joy,0.163719
1918,Sorry. This thing knows whats up!,remorse,0.147592


In [160]:
df_metric[['text', 'original_label', 'normalized_entropy']].to_csv(output_path)